In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix

In [ ]:
vertices = pd.read_csv('vertices.csv')
edges = pd.read_csv('edges.csv')
ids = pd.read_csv('ids.csv')
new_vertices = pd.read_csv('new_vertices.csv')

In [ ]:
# building sparse matrix
def build_matr():
    A = lil_matrix((len(vertices), len(vertices)))
    for row in edges.values:
        A[int(row[0])-1, int(row[1])-1] = 1 
        A[int(row[1])-1, int(row[0])-1] = 1 
    return(A)

In [ ]:
A = build_matr()

### Training SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=450,algorithm='randomized', random_state=42)
transformed = svd.fit_transform(A)
components = svd.components_

In [ ]:
np.save('components.npy', components)
np.save('transformed.npy', transformed)

### Weights assigning

In [ ]:
summa = new_vertices[new_vertices.id.isin(ids.id)].count_connections.sum()
df = ids.merge(new_vertices[['id', 'count_connections']], on='id')
df['k'] = df.count_connections/summa
df['k']*=100000

### hop 2-3  and search functions

In [ ]:
#preprocessed data here https://drive.google.com/file/d/1a0Gbzp1U251twbcr6MUuqrOBIY4bB-Bz/view
all_pairs = pd.read_csv('all_connections')
index = all_pairs.id_1
connections = all_pairs.connection
inc = dict(zip(index, map(eval, connections)))

def BFS1(root):
    visited = set()
    visited.add(root)
    res = set()
    for neighbour in inc[root]:
         if neighbour not in visited: 
            res.add(neighbour)
    return(visited, res)
def BFS2(visited, q):
    res = set()
    for i in q:
        visited.add(i)
        for neighbour in inc[i]:
             if neighbour not in visited: 
                    res.add(neighbour)
    return(visited, res)
def BFS3(visited, q):
    res = set()
    for i in q:
        visited.add(i)
        for neighbour in inc[i]:
             if neighbour not in visited: 
                    res.add(neighbour)
    return(visited, res)
def hop3(root):
    v, q = BFS1(root)
    v,q = BFS2(v,q)
    v,q = BFS3(v,q)
    return(q)
def hop2(root):
    v, q = BFS1(root)
    v,q = BFS2(v,q)
    return(q)

In [ ]:
import tqdm

### Predicting

In [ ]:
itog = pd.DataFrame({'id_1':[], 'id_2':[], 'prob':[]})
for i in tqdm.tqdm(ids.id):
    k = df[df.id==i].k.sum()*100000
    q = i-1
    check = new_vertices[(new_vertices['id'].isin(hop3(i)) | new_vertices['id'].isin(hop2(i)))].id
    probs = transformed[q]@components[:,check-1]
    probs= abs(probs-1)
    res = pd.DataFrame({'id_2':check, 'prob':probs})
    res['id_1'] = i
    res['min'], res['max'] = res[['id_1', 'id_2']].min(axis=1),res[['id_1', 'id_2']].max(axis=1) 
    res = res.drop(columns=['id_1', 'id_2']).rename(columns={'min':'id_1', 'max':'id_2'}).sort_values('prob')
    itog = itog.append(res[['id_1', 'id_2', 'prob']][:int(1.5*k)+1])

In [ ]:
def right_form(matr):
    return(edges[['id_1', 'id_2']].append(matr[['id_1', 'id_2']], sort=False).drop_duplicates()[len(edges):])
    

In [ ]:
right_form(itog).drop_duplicates().astype(int)[:100000].to_csv('svdResult.csv', index=False)